1) 자치단체: 서울, 인천, 대전, 세종, 광주, 대구, 부산, 울산, 제주  & 본청  '검색'
2) 해당지역 자치법규 목록 클릭
3) 저장 클릭 
-> 최종적으로 9개 엑셀파일이 저장됨

1) 자치단체: 경기, 강원, 충남, 충북, 경남, 경북, 전남, 전북  & 본청을 제외한 모든 시군  '검색'
2) 해당지역 자치법규 목록 클릭
3) 저장 클릭 
-> 최종적으로 152개 엑셀파일이 저장됨

In [78]:
import os
import time
from tqdm import tqdm
import shutil
import requests
import urllib.request
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
# options.add_argument('window-size=1200x600')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)

<ipython-input-6-519b71479459>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
<ipython-input-6-519b71479459>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


In [72]:
#{ctpvCd: null, sggCd: "000", ctpvNm: null, sggNm: "본청"}
# 본청코드는 000임
si_dict = {"서울":11,"부산":26,"대구":27,"인천":28,"광주":29,"대전":30,"울산":31,"세종":36,"제주":50}
do_dict = {"경기":41,"강원":42,"충북":43,"충남":44,"전북":45,"전남":46,"경북":47,"경남":48}

# 세종은 본청코드가 110임 파일의 순서를 위해 잘못저장된 "해당 지역자치법규 리스트_20230505_012305"를 새로 받아서 저장

In [73]:
def do_si_code_dict_from_file(json_path)-> dict:
    si_info_dict = {}

    if not os.path.isfile(json_path):
        return None
    
    with open(json_path,'r') as json_file:
        # set이 안맞는 경우는 없음
        ssgCd = None
        ssgNm = None

        while True:
            line = json_file.readline()
            if not line: break
            if "sggCd" in line:
                ssgCd = line.split(":")[1].replace('"', '').replace(",", "").strip()
            if "sggNm" in line:
                ssgNm = line.split(":")[1].replace('"', '').replace(",", "").strip()

            if "}" in line:
                # key value add
                if ssgNm == "본청": continue
                si_info_dict[ssgNm] = ssgCd
    return si_info_dict

In [82]:
for code_name, code in tqdm(si_dict.items()):
    driver.get("https://www.elis.go.kr/alrpop/locgovAlrPopup?ctpvCd="+str(code)+"&sggCd=000")
    driver.find_element(By.XPATH, '//*[@id="btnExcel"]').click()
    time.sleep(5)

100%|██████████| 9/9 [00:57<00:00,  6.34s/it]


In [85]:
total_cnt = 0
for do_code_name, do_code in do_dict.items():
    do_si_dict = do_si_code_dict_from_file("./code/"+do_code_name+".json")
    if do_si_dict == None:
        print(do_code_name + "의 정보가 없습니다.")
        continue
    # 도별로 폴더 만들기
    #if not os.path.exists("./"+do_code_name):
    #    os.mkdir("./"+do_code_name)
        
    for si_code_name, si_code in tqdm(do_si_dict.items()):
        total_cnt += 1
        driver.get("https://www.elis.go.kr/alrpop/locgovAlrPopup?ctpvCd="+str(do_code)+"&sggCd="+str(si_code))
        driver.find_element(By.XPATH, '//*[@id="btnExcel"]').click()
        time.sleep(3)

print(total_cnt)

100%|██████████| 18/18 [01:38<00:00,  5.49s/it]

152
